# Inventory Forecast Optimization Project
This notebook demonstrates time series forecasting and KPI tracking using simulated supply chain data for inventory and demand forecasting.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
# TensorFlow imports (to be run in local/Colab)
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense


## Load and Preview Data

In [ ]:
# Load inventory and demand datasets
inventory_df = pd.read_csv('inventory_data.csv')
demand_df = pd.read_csv('demand_signals.csv')

# Preview datasets
display(inventory_df.head())
display(demand_df.head())


## Merge Inventory and Demand Data for Analysis

In [ ]:
# Focus on one product for demonstration
product_focus = 'A1'
merged_df = pd.merge(demand_df, inventory_df, on='product_id')
product_df = merged_df[merged_df['product_id'] == product_focus].copy()
product_df['week'] = pd.to_datetime(product_df['week'])
product_df.sort_values('week', inplace=True)


## Simulate Inventory Movement and Stockouts

In [ ]:
product_df['inventory_level'] = product_df['current_inventory'].iloc[0]
product_df['order_placed'] = False
product_df['stockout'] = False

for i in range(1, len(product_df)):
    demand = product_df.iloc[i]['demand_forecast']
    prev_inventory = product_df.iloc[i-1]['inventory_level']
    reorder_point = product_df.iloc[i]['reorder_point']
    lead_time = product_df.iloc[i]['lead_time_days']
    
    inventory_level = prev_inventory - demand
    if inventory_level < 0:
        inventory_level = 0
        product_df.loc[product_df.index[i], 'stockout'] = True

    if inventory_level <= reorder_point and not product_df.iloc[i]['order_placed']:
        arrival_index = i + lead_time if i + lead_time < len(product_df) else -1
        product_df.loc[product_df.index[arrival_index], 'inventory_level'] += product_df.iloc[i]['safety_stock']
        product_df.loc[product_df.index[i], 'order_placed'] = True

    product_df.loc[product_df.index[i], 'inventory_level'] = inventory_level


## Calculate Key Performance Indicators (KPIs)

In [ ]:
stockout_rate = product_df['stockout'].mean()
average_inventory = product_df['inventory_level'].mean()
reorder_frequency = product_df['order_placed'].sum()
service_level = 1 - stockout_rate

kpi_summary = pd.DataFrame({
    'KPI': ['Stockout Rate', 'Average Inventory Level', 'Reorder Frequency', 'Service Level'],
    'Value': [f"{stockout_rate:.2%}", f"{average_inventory:.2f} units", reorder_frequency, f"{service_level:.2%}"],
    'Definition': [
        "Percentage of weeks when demand could not be fully met due to insufficient inventory.",
        "Average inventory units available across all weeks.",
        "Total number of times a reorder was triggered due to inventory falling below reorder point.",
        "Percentage of weeks with no stockouts—indicates how well demand was served."
    ]
})
display(kpi_summary)


## Visualize KPIs

In [ ]:
plt.figure(figsize=(10, 6))
kpi_values_fixed = [float(v.strip('%').replace(' units', '')) if isinstance(v, str) else float(v) for v in kpi_summary['Value']]
bars = plt.bar(kpi_summary['KPI'], kpi_values_fixed, color=['red', 'blue', 'green', 'orange'])
plt.title('Key Performance Indicators (KPIs) for Product A1')
plt.ylabel('Value')
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.5, f'{yval:.2f}', ha='center', va='bottom')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## Forecast Demand using ARIMA

In [ ]:
demand_series = product_df.set_index('week')['demand_forecast']
train_size = int(len(demand_series) * 0.8)
train, test = demand_series[:train_size], demand_series[train_size:]

model = ARIMA(train, order=(1, 1, 1))
model_fit = model.fit()
forecast = model_fit.forecast(steps=len(test))
rmse = np.sqrt(mean_squared_error(test, forecast))

plt.figure(figsize=(14, 6))
plt.plot(train.index, train, label='Training Data')
plt.plot(test.index, test, label='Actual Demand (Test)', color='green')
plt.plot(test.index, forecast, label='ARIMA Forecast', color='red', linestyle='--')
plt.title(f'ARIMA Demand Forecast for Product A1\nRMSE: {rmse:.2f}')
plt.xlabel('Week')
plt.ylabel('Forecasted Demand')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## (Optional) Forecast Demand using LSTM
> Note: This section requires TensorFlow to be installed. Run in Google Colab or local Python environment.

In [ ]:
# Uncomment to use:
# lstm_data = demand_series.values.reshape(-1, 1)
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(lstm_data)

# def create_sequences(data, lookback=4):
#     X, y = [], []
#     for i in range(len(data) - lookback):
#         X.append(data[i:i + lookback])
#         y.append(data[i + lookback])
#     return np.array(X), np.array(y)

# lookback = 4
# X, y = create_sequences(scaled_data, lookback)
# split_index = int(0.8 * len(X))
# X_train, X_test = X[:split_index], X[split_index:]
# y_train, y_test = y[:split_index], y[split_index:]

# model = Sequential()
# model.add(LSTM(units=50, activation='relu', input_shape=(lookback, 1)))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mean_squared_error')
# model.fit(X_train, y_train, epochs=50, batch_size=4, verbose=0)

# y_pred = model.predict(X_test)
# y_pred_inverse = scaler.inverse_transform(y_pred)
# y_test_inverse = scaler.inverse_transform(y_test)

# test_weeks = product_df['week'].iloc[lookback + split_index:lookback + split_index + len(y_pred)]
# plt.figure(figsize=(14, 6))
# plt.plot(test_weeks, y_test_inverse, label='Actual Demand (Test)', color='green')
# plt.plot(test_weeks, y_pred_inverse, label='LSTM Forecast', color='purple', linestyle='--')
# plt.title('LSTM Demand Forecast for Product A1')
# plt.xlabel('Week')
# plt.ylabel('Forecasted Demand')
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()
